<a href="https://colab.research.google.com/github/Arminsbss/Reinforcement-learning-N-N-grid/blob/main/reinforcement_learning_n_n_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load spreadsheet
Grid = pd.read_excel('Grid.xlsx',header=None)
Grid = Grid.values


In [ ]:
Grid[0]


array([ -5,  -4,   6,   0,  14,  -4,  -2,   6,  14,   2,  12,   6,   3,
        -1, -10,  -4, -10,   8,   4,  -4,  -5, -10,   2,  -3,  10,  -2,
        -8,  -1,   7,  25])